In [1]:
import pandas as pd
import datetime as dt
import re as re

In [2]:
#the event is depending on the barcode scanned afterwoards

def activity_dependency(barcode):
    switcher = {
        'Production Start': 'Factory Closed',
        'New Recipe Start': 'Preparation/Changeover',
        'Break End': 'Break',
        'Production Finish': 'TBD'
    }
    return switcher.get(barcode, 'Production')

In [3]:
# 7 days of Assembly

def batchcode_dependency(batchcoded):
    switcher = {
        1: 'Friday Assembly',
        2: 'Saturday Assembly',
        3: 'Sunday Assembly',
        4: 'Monday Assembly',
        5: 'Tuesday Assembly',
        6: 'Wednesday Assembly',
        7: 'Thursday Assembly'
    }
    return switcher.get(batchcoded, '')

In [4]:
# morning shift -- 5:00 to 14:00
# afternoon shift -- 14:00 to 23:00
# Error -- Events compeleted in other time periods

def categorize_event_shift(timestamp):
    weekday_index = pd.Timestamp(timestamp).weekday()
    switcher = {
        0: 'Monday',
        1: 'Tuesday',
        2: 'Wednesday',
        3: 'Thursday',
        4: 'Friday',
        5: 'Saturday',
        6: 'Sunday'
    }
    weekday = switcher.get(weekday_index,'')
    hour = pd.Timestamp(timestamp).hour
    if hour in range(5,14):
        shift = 'Morning'
    elif hour in range(14,23):
        shift = 'Afternoon'
    else:
        shift = 'Error'
    return weekday +' '+ shift

In [5]:
#calculting time cost to sec

def time_consumption(timestamp1,timestamp2):
    t1 = pd.to_datetime(timestamp1)
    t2 = pd.to_datetime(timestamp2)
    InMin = pd.Timedelta(t2-t1).seconds/60.0
    return format(InMin, '.2f')


In [6]:
# count how many pickers for ever events (+2 to get the total amount of people on this kitting line)

def count_pickers(picker):
    pattern = re.compile(r';|_|,')  ## <- temporary solutions overhere, may change according to the labels printed
    picker_array = pattern.findall(picker)  
    return len(picker_array)

In [7]:
staged_df = pd.read_csv(r'C:\Users\Peiran Quan\Desktop\python_data_preparation\staged.csv')  # Can change file path here
staged_df.head()

,Barcode,Production Batch,Recipe and P,Timestamp,Date,Seq Code,Week,Team Leader,Replenisher,Pickers,Break Reasons,Missing Products,Kitting Line,Timestamp and Date
0,Production Start,NaN,NaN,06:10:05,2020-12-03 00:00:00,NaN,NaN,iXUsr_Siregar(6092),NaN,iXUsr_Amaya(9984)iXUsr_Wijaya(9333),NaN,NaN,KL1,2020-12-03 06:10:05
1,New Recipe Start,NaN,NaN,06:14:00,2020-12-03 00:00:00,NaN,NaN,iXUsr_Siregar(6092),NaN,iXUsr_Amaya(9984)iXUsr_Wijaya(9333),NaN,NaN,KL1,2020-12-03 06:14:00
2,650F4$$$$$$$$0001000701,6.0,F4,06:14:13,2020-12-03 00:00:00,0001 of 0007,50.0,iXUsr_Siregar(6092),NaN,iXUsr_Amaya(9984)iXUsr_Wijaya(9333),NaN,NaN,KL1,2020-12-03 06:14:13
3,650F4$$$$$$$$0002000701,6.0,F4,06:16:49,2020-12-03 00:00:00,0002 of 0007,50.0,iXUsr_Siregar(6092),NaN,iXUsr_Amaya(9984)iXUsr_Wijaya(9333),NaN,NaN,KL1,2020-12-03 06:16:49
4,650F4$$$$$$$$0003000701,6.0,F4,06:20:08,2020-12-03 00:00:00,0003 of 0007,50.0,iXUsr_Siregar(6092),NaN,iXUsr_Amaya(9984)iXUsr_Wijaya(9333),NaN,NaN,KL1,2020-12-03 06:20:08


In [8]:
# copy raw df
df_final = pd.DataFrame(data=None) #df is the event data frame

In [9]:
def main(df_temp_raw):
    df_temp_event = pd.DataFrame(columns=('Start Time', 'Finish Time', 'Activity', 'Seq Code' ,'Recipe Name', 'Break Reasons', 'Missing Ingredients', 'Kitting Line', 'Assembly Batch', 'Event Shift', 'Team Leader', 'Pickers Count', 'Time Consumption'))
    index_align = df_temp_raw.first_valid_index() # pandas is using the df_temp_raw's frame index whenever df_temp_raw is called
    df_temp_event['Finish Time'] = df_temp_raw['Timestamp and Date']
    df_temp_event['Seq Code'] = df_temp_raw['Seq Code']
    df_temp_event['Start Time'].loc[index_align] = '2019-1-1 00:00:00'
    #df_temp_event['Recipe Name'][1] = df_temp_raw['Recipe and P'].loc[df_temp_raw['Recipe and P'].first_valid_index()]
    #print(df_temp_event)
    df_temp_raw.loc[index_align, 'Recipe and P'] = df_temp_raw['Recipe and P'].loc[df_temp_raw['Recipe and P'].first_valid_index()] #ignore the warning -- data type is not changing, the return is in VIEW

    for i in range (len(df_temp_raw)):
        df_temp_event['Start Time'].loc[index_align+i+1] = df_temp_raw['Timestamp and Date'].loc[index_align+i]
        df_temp_event['Activity'].loc[index_align+i] = activity_dependency(df_temp_raw['Barcode'].loc[index_align+i])
        df_temp_event['Break Reasons'].loc[index_align+i+1] = df_temp_raw['Break Reasons'].loc[index_align+i]
        df_temp_event['Missing Ingredients'].loc[index_align+i+1] = df_temp_raw['Missing Products'].loc[index_align+i]
        df_temp_event['Kitting Line'].loc[index_align+i+1] = df_temp_raw['Kitting Line'].loc[index_align+i]
        df_temp_event['Assembly Batch'].loc[index_align+i] = batchcode_dependency(df_temp_raw['Production Batch'].loc[index_align+i])
        df_temp_event['Time Consumption'].loc[index_align+i] = time_consumption(df_temp_event['Start Time'].loc[index_align+i], df_temp_event['Finish Time'].loc[index_align+i])
        df_temp_event['Event Shift'].loc[index_align+i] = categorize_event_shift(df_temp_event['Finish Time'].loc[index_align+i])
        df_temp_event['Team Leader'].loc[index_align+i] = df_temp_raw['Team Leader'].loc[index_align+i]
        df_temp_event['Pickers Count'].loc[index_align+i] = count_pickers(df_temp_raw['Pickers'].loc[index_align+i])
        #print(df_temp_event)
        """
        for j in range (len(df_temp_raw)):  #############################################
            print(df_temp_raw['Recipe and P'][::-1].loc[(index_align + j):])
            reversed_index = df_temp_raw['Recipe and P'][::-1].loc[(index_align + j):].first_valid_index()
        """
        #df_temp_raw['Recipe and P'][index_align] = df_temp_raw['Recipe and P'].loc[df_temp_raw['Recipe and P'].first_valid_index()]
        recipe_index = df_temp_raw['Recipe and P'].loc[0:index_align+i].last_valid_index()
        #print(recipe_index)
        df_temp_event['Recipe Name'].loc[index_align+i] = df_temp_raw['Recipe and P'].loc[recipe_index]
        #print(df_temp_raw['Recipe and P'].loc[recipe_index])
    #print(df_temp_event)
    return df_temp_event

In [10]:
for kl in range(1,21):
    df_temp_raw = staged_df.loc[staged_df['Kitting Line'] == 'KL%s'%kl]
    if df_temp_raw.empty:
        print("Kitting Line %r is not included!" %(kl))  # if data frame is empty
    #print(df_temp_raw)
    else:
        seg = main(df_temp_raw)
        #print(seg)
        df_final = pd.concat([df_final, seg], sort = False)
        print("Kitting Line %r Completed!" %(kl))


Kitting Line 1 Completed!
Kitting Line 2 Completed!
Kitting Line 3 Completed!
Kitting Line 4 Completed!
Kitting Line 5 Completed!
Kitting Line 6 Completed!
Kitting Line 7 Completed!
Kitting Line 8 Completed!
Kitting Line 9 Completed!
Kitting Line 10 Completed!
Kitting Line 11 Completed!
Kitting Line 12 Completed!
Kitting Line 13 Completed!
Kitting Line 14 Completed!
Kitting Line 15 Completed!
Kitting Line 16 Completed!
Kitting Line 17 Completed!
Kitting Line 18 Completed!
Kitting Line 19 Completed!
Kitting Line 20 Completed!


In [11]:
## Attach a column to identify the Week
## Update weekly
df_final['Week'] = '2020-51'
df_final.tail()

,Start Time,Finish Time,Activity,Seq Code,Recipe Name,Break Reasons,Missing Ingredients,Kitting Line,Assembly Batch,Event Shift,Team Leader,Pickers Count,Time Consumption,Week
6384,2020-12-06 19:27:31,2020-12-06 19:28:37,Production,NaN,K2,NaN,NaN,KL20,,Sunday Afternoon,iXUsr_Kurnia(6899),3,1.10,2020-50
6385,2020-12-06 19:28:37,2020-12-06 19:29:52,Break,NaN,K2,missing products,"Dill & Parsley Mayonnaise, 50g",KL20,,Sunday Afternoon,iXUsr_Kurnia(6899),3,1.25,2020-50
6386,2020-12-06 19:29:52,2020-12-06 19:32:04,Production,0007 of 0008,K2,NaN,NaN,KL20,Tuesday Assembly,Sunday Afternoon,iXUsr_Kurnia(6899),3,2.20,2020-50
6387,2020-12-06 19:32:04,2020-12-06 19:35:31,Production,0008 of 0008,K2,NaN,NaN,KL20,Tuesday Assembly,Sunday Afternoon,iXUsr_Kurnia(6899),3,3.45,2020-50
6388,2020-12-06 19:35:31,2020-12-06 20:54:07,TBD,NaN,K2,NaN,NaN,KL20,,Sunday Afternoon,iXUsr_Kurnia(6899),3,78.60,2020-50


In [12]:
df_final.to_csv(r'C:\Users\Peiran Quan\Desktop\python_data_preparation\prepared.csv', index = False) # Can change file path here